## Total edge에 user edge만 추가

In [1]:
from networkx.algorithms import bipartite
import networkx as nx
from networkx import *
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 80000row
user_prefer = pd.read_csv('ml-100k/u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_prefer = user_prefer[['userID','itemID','rating']]

# 100000row
user_cluster = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_cluster = user_cluster[['userID','itemID','rating']]

# 20000row
user_test = pd.read_csv('ml-100k/u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]


In [3]:
'''
#print(train.shape)
#print(test.shape)
train_list = []
test_list = []

import numpy as np
from sklearn.model_selection import RepeatedKFold

user_prefer = pd.read_csv('ml-1m/ratings.dat', sep='::',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_prefer = user_prefer[['userID','itemID','rating']]

random_state = 12883823
rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=random_state)
for train, test in rkf.split(user_prefer):
    result = next(rkf.split(user_prefer), None)
    train = user_prefer.iloc[result[0]]
    test =  user_prefer.iloc[result[1]]
    
    train_list.append(train)
    test_list.append(test)
    print("%s %s" % (train.shape, test.shape))
    '''

'\n#print(train.shape)\n#print(test.shape)\ntrain_list = []\ntest_list = []\n\nimport numpy as np\nfrom sklearn.model_selection import RepeatedKFold\n\nuser_prefer = pd.read_csv(\'ml-1m/ratings.dat\', sep=\'::\',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)\nuser_prefer = user_prefer[[\'userID\',\'itemID\',\'rating\']]\n\nrandom_state = 12883823\nrkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=random_state)\nfor train, test in rkf.split(user_prefer):\n    result = next(rkf.split(user_prefer), None)\n    train = user_prefer.iloc[result[0]]\n    test =  user_prefer.iloc[result[1]]\n    \n    train_list.append(train)\n    test_list.append(test)\n    print("%s %s" % (train.shape, test.shape))\n    '

In [4]:
'''
# 800167row
user_prefer = train_list[0]

# 1000209row
user_cluster = pd.read_csv('ml-1m/ratings.dat', sep='::',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_cluster = user_cluster[['userID','itemID','rating']]

# 200042row
user_test = train_list[0]
'''

'\n# 800167row\nuser_prefer = train_list[0]\n\n# 1000209row\nuser_cluster = pd.read_csv(\'ml-1m/ratings.dat\', sep=\'::\',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)\nuser_cluster = user_cluster[[\'userID\',\'itemID\',\'rating\']]\n\n# 200042row\nuser_test = train_list[0]\n'

In [5]:

min_max_scaler = MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(user_prefer[['rating']])
user_prefer[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_cluster[['rating']])
user_cluster[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_test [['rating']])
user_test [['rating']] = x_scaled


In [6]:
user_name_cluster = user_cluster.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_artist_rating = user_prefer.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0) # train 
user_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0) # test

#idx = user_name_cluster.index

In [7]:
## user-item node 이름 설정

matrix = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
matrix[['userID']] = 'u' + matrix[['userID']].astype(str)
matrix[['itemID']] = 'i' + matrix[['itemID']].astype(str)
matrix

,userID,itemID,rating,timestamp
0,u196,i242,3,881250949
1,u186,i302,3,891717742
2,u22,i377,1,878887116
3,u244,i51,2,880606923
4,u166,i346,1,886397596
...,...,...,...,...
99995,u880,i476,3,880175444
99996,u716,i204,5,879795543
99997,u276,i1090,1,874795795
99998,u13,i225,2,882399156


In [8]:
import numpy as np

user_node = matrix[['userID']].values
user_node = np.array(user_node).flatten().tolist()

#user_node

In [9]:
G = nx.Graph()
G.add_nodes_from(user_node)
info(G)

C:\Users\kddin\AppData\Local\Temp\ipykernel_16776\1259828340.py:3: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  info(G)


'Graph with 943 nodes and 0 edges'

In [10]:
edgelist = []

for i in matrix.values:
    edgelist.append((i[0],i[1]))
    
B = nx.Graph() # >300
B.add_nodes_from(matrix.userID, bipartite=0) # user
B.add_nodes_from(matrix.itemID, bipartite=1) # item(movie)
B.add_edges_from(edgelist)


info(B)

C:\Users\kddin\AppData\Local\Temp\ipykernel_16776\3805399103.py:12: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  info(B)


'Graph with 2625 nodes and 100000 edges'

### 최대 연결 엣지 444153 (=943*(942/2)

In [11]:
for i in range(10):
    user_edgelist = []
    total_edgelist = []
    pred_link = list(nx.common_neighbor_centrality(B, alpha=0.8))
    
    link_likelihood = pd.DataFrame(pred_link).iloc[:,2].values
    link_95 = np.percentile(link_likelihood,  95, interpolation='linear')
    #link_mean = np.mean(link_likelihood)
    
    for p in pred_link:
        edge = list(p)
        linklihood = int(edge[2])
        
        if linklihood > link_95:
            if edge[0][:1] ==  edge[1][:1] == 'u': # user R
                user_edgelist.append((edge[0], edge[1]))
                total_edgelist.append((edge[0], edge[1]))
            else :
                total_edgelist.append((edge[0], edge[1]))
                
            
    B.add_edges_from(total_edgelist)
    G.add_edges_from(user_edgelist)
    
    print('total_graph connected : ', is_connected(B),' user graph info : ', info(B))
    print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))
    
    if is_connected(B):
        break
    
#print('total graph: ', info(B))
#print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))

C:\Users\kddin\AppData\Local\Temp\ipykernel_16776\4127674531.py:7: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  link_95 = np.percentile(link_likelihood,  95, interpolation='linear')


total_graph connected :  True  user graph info :  Graph with 2625 nodes and 266944 edges
user_graph connected :  False  user graph info :  Graph with 943 nodes and 80764 edges


C:\Users\kddin\AppData\Local\Temp\ipykernel_16776\4127674531.py:25: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('total_graph connected : ', is_connected(B),' user graph info : ', info(B))
C:\Users\kddin\AppData\Local\Temp\ipykernel_16776\4127674531.py:26: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))


In [12]:
## network clustering

def graph_clustering(cluster_num):
    from networkx.algorithms import community
    cluster = [0]*cluster_num
    
    for i, community in enumerate(community.asyn_fluidc(B, k=cluster_num)):
        cluster[i]=(community)
                #print("community_{:0>2d}: {}".format(i, community))
                
    
    '''
    if nx.is_connected(G1): 
        for k in range(cluster_num, cluster_num+1):
            for i, community in enumerate(community.asyn_fluidc(B, k=k)):
                #print("community_{:0>2d}: {}".format(i, community))
                cluster[i]=(community)
        # print("="*40)
        # print(info(B))
        

    #else:
        #print("G가 not connected이면 실행안됨")
        '''
        
    ## user cluster 정보
    user_c = pd.read_csv('ml-100k/u.user', sep='|',names=["userID","age","gender","occupation","zip code"],header=None,na_filter=False)
    user_c = user_c[['userID']]
    user_c = user_c.set_index('userID')
    user_c['cluster'] = 100
    
    '''
    # 각 cluster에 속한 user, item 의 수
    for i in range(cluster_num):
        print(i,'번째 cluster:' , len(cluster[i]))
        '''
        
    # 각 cluster에 속한 user의 수 
    for i in range(cluster_num):
        cnt = 0
        for j in list(cluster[i]):
            if j[0] == 'u':
                cnt += 1
                user_c.iloc[int(j[1:])-1] = i 
            
        #print(i,'번째 cluster user:' , cnt)
    
    #print(user_c)
        
    return (user_c)
        

In [13]:
import numpy as np

def grs(cluster_num, user_artist_rating):

    #graph_clustering(cluster_num)
    normalization_user_artist_rating = user_artist_rating
    kcluster = graph_clustering(cluster_num) # userID - cluster


    normalization_user_artist_rating["sum"] = normalization_user_artist_rating.mean(axis=1)  # train 
    user_test["sum"] = user_test.mean(axis=1) # test

    #train, test셋에 cluster 할당
    cluster = normalization_user_artist_rating 
    cluster["cluster"] = np.nan # train
    user_test["cluster"] = np.nan # test

    for i in kcluster.index:
        if i in cluster.index:
            cluster["cluster"][i] = kcluster["cluster"][i] # train
        if i in user_test.index:
            user_test["cluster"][i] = kcluster["cluster"][i] # test

    cluster_user_matrix = pd.DataFrame(cluster)

    # (user_test)cluster 수 및 cluster별 인원 수 저장하는 list
    length = [1]*cluster_num
    for i in range(cluster_num):
        length[i] = sum(user_test["cluster"] == i)
    #length.append(user_test["cluster"].value_counts())
    #print(length)

    clusters = []

    # user-item 정보 클러스터 별로 저장
    for i in range(len(length)):
        clusters.append(cluster_user_matrix[cluster_user_matrix["cluster"]==i])
        
    sum_point_idx = []
    for i in range(len(length)):
        sum_point_idx.append(i)
    
    # predict 저장할 DataFrame
    sum_point = pd.DataFrame(index=sum_point_idx, columns = cluster_user_matrix.columns).fillna(0)

    for i in range(cluster_num):
        # clusters[i] = clusters[i].replace({'0':np.nan, 0:np.nan})
        #sum_point.loc[i] = clusters[i].max(axis=0, skipna=True)
        sum_point.loc[i] = clusters[i].mean(axis=0, skipna=True)
    
    sum_point = sum_point.replace({np.nan:0})

    point = sum_point[sum_point.index < len(length)]

    scores = point.drop(["cluster"], axis=1)
    cluster_user_matrix.drop(["sum"], axis=1)

    # 정답 셋
    tmp = pd.DataFrame(columns = user_test.columns)
    tmp["num"] = np.nan

    # 정답 셋 정보 할당
    for i in user_test.index:
        idx = 0
        while idx < len(length):
            if user_test["cluster"][i] == idx:
                tmp.loc[i] = user_test.loc[i]
                tmp["num"][i] = idx
            idx += 1

    number = pd.DataFrame(tmp["num"])

    # 평가
    from sklearn.metrics import ndcg_score

    tmp = tmp.drop(["sum","cluster","num"], axis=1)

    # miss matching 제거
    for i in scores.columns:
        if i not in tmp.columns:
            scores = scores.drop([i], axis=1)

    for i in tmp.columns:
        if i not in scores.columns:
            tmp = tmp.drop([i], axis=1)
        
    # ndcg
    result = []
    for i in range(len(length)):
        result.append(0)

    for i in tmp.index:
        idx = 0

        while idx < len(length):
            if number["num"][i] == idx:
                result[idx] += ndcg_score([tmp.loc[i]], [scores.loc[idx]])
            idx += 1

    for i in range(len(length)):
        if length[i] >0:
            result[i] = result[i]/length[i]
        else:
            result[i] = 0

    print("cluster수:",len(length),"/ NDCG:",'%.5f'%(sum(result)/(len(length))))
    #print("cluster수:",len(length),"/ NDCG:",'%.5f'%(sum(result)/(len(length))))

In [14]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import numpy as np

cluster_num = range(2,51)
#cluster_num = [2]

for i in cluster_num:
    #matrix = SVD.fit_transform(user_name_cluster)
    #matrix = pd.DataFrame(matrix)
    #matrix.index = idx
    user_artist_rating = pd.DataFrame(user_artist_rating)
    ms = MinMaxScaler()
    #ccc = pd.DataFrame(ms.fit_transform(close_centrality_drop))
    #ccc.index = close_centrality_drop.index
    #ccc.columns = close_centrality_drop.columns

    grs(i, user_artist_rating)

cluster수: 2 / NDCG: 0.49656
cluster수: 3 / NDCG: 0.47125
cluster수: 4 / NDCG: 0.37929
cluster수: 5 / NDCG: 0.38440
cluster수: 6 / NDCG: 0.45659
cluster수: 7 / NDCG: 0.48243
cluster수: 8 / NDCG: 0.43922
cluster수: 9 / NDCG: 0.42639
cluster수: 10 / NDCG: 0.43896
cluster수: 11 / NDCG: 0.43425
cluster수: 12 / NDCG: 0.39080
cluster수: 13 / NDCG: 0.40179
cluster수: 14 / NDCG: 0.42099
cluster수: 15 / NDCG: 0.41749
cluster수: 16 / NDCG: 0.36752
cluster수: 17 / NDCG: 0.44758
cluster수: 18 / NDCG: 0.39971
cluster수: 19 / NDCG: 0.42835
cluster수: 20 / NDCG: 0.41011
cluster수: 21 / NDCG: 0.37525
cluster수: 22 / NDCG: 0.43017
cluster수: 23 / NDCG: 0.37231
cluster수: 24 / NDCG: 0.34796
cluster수: 25 / NDCG: 0.39354
cluster수: 26 / NDCG: 0.43397
cluster수: 27 / NDCG: 0.40434
cluster수: 28 / NDCG: 0.34057
cluster수: 29 / NDCG: 0.38019
cluster수: 30 / NDCG: 0.39974
cluster수: 31 / NDCG: 0.40713
cluster수: 32 / NDCG: 0.39887
cluster수: 33 / NDCG: 0.37465
cluster수: 34 / NDCG: 0.35287
cluster수: 35 / NDCG: 0.35965
cluster수: 36 / NDCG: 0

In [15]:
max 사용 시 
cluster수: 2 / NDCG: 0.34329
cluster수: 3 / NDCG: 0.35080
cluster수: 4 / NDCG: 0.34621
cluster수: 5 / NDCG: 0.33781
cluster수: 6 / NDCG: 0.35343
cluster수: 7 / NDCG: 0.34394
cluster수: 8 / NDCG: 0.34468
cluster수: 9 / NDCG: 0.34175
cluster수: 10 / NDCG: 0.34943
cluster수: 11 / NDCG: 0.36301
cluster수: 12 / NDCG: 0.34179
cluster수: 13 / NDCG: 0.32174
cluster수: 14 / NDCG: 0.32101
cluster수: 15 / NDCG: 0.36600
cluster수: 16 / NDCG: 0.33584
cluster수: 17 / NDCG: 0.38261
cluster수: 18 / NDCG: 0.32312
cluster수: 19 / NDCG: 0.30543
cluster수: 20 / NDCG: 0.31864
        
mean 사용시
cluster수: 2 / NDCG: 0.49588
cluster수: 3 / NDCG: 0.50117
cluster수: 4 / NDCG: 0.44782
cluster수: 5 / NDCG: 0.48748
cluster수: 6 / NDCG: 0.41585
cluster수: 7 / NDCG: 0.45589
cluster수: 8 / NDCG: 0.44635
cluster수: 9 / NDCG: 0.46367
cluster수: 10 / NDCG: 0.42188
cluster수: 11 / NDCG: 0.41784
cluster수: 12 / NDCG: 0.44745
cluster수: 13 / NDCG: 0.40972
cluster수: 14 / NDCG: 0.42372
cluster수: 15 / NDCG: 0.37773
cluster수: 16 / NDCG: 0.42483
cluster수: 17 / NDCG: 0.39562
cluster수: 18 / NDCG: 0.42812
cluster수: 19 / NDCG: 0.41915
cluster수: 20 / NDCG: 0.38772

        

SyntaxError: invalid syntax (796871815.py, line 1)